<a href="https://colab.research.google.com/github/WMCh/AccessorySimulator/blob/main/AccessorySimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Define some constants and probabilities.

In [1]:
NAME = {
    'ring': '戒指',
    'earring': '耳饰',
    'accessory': '配饰',
    'bracelet': '手镯'
}

LEVEL_POOL = [60, 80, 100, 120, 140, 160]

PIC_DIC = {
    'ring': {
        60: None,
        80: None,
        100: None,
        120: None,
        140: None,
        160: None
    },
    'earring': {
        60: None,
        80: None,
        100: None,
        120: None,
        140: None,
        160: None
    },
    'accessory': {
        60: None,
        80: None,
        100: None,
        120: None,
        140: None,
        160: None
    },
    'bracelet': {
        60: None,
        80: None,
        100: None,
        120: None,
        140: None,
        160: None
    }
}

NAME_DIC = {
    'ring': {
        60: '枫华戒',
        80: '芙蓉戒',
        100: '金鳞绕',
        120: '悦碧水',
        140: '九曜光华',
        160: '太虚渺云'
    },
    'earring': {
        60: '翠叶环',
        80: '明月珰',
        100: '玉蝶翩',
        120: '点星芒',
        140: '凤羽流苏',
        160: '焰云霞珠'
    },
    'accessory': {
        60: '芝兰佩',
        80: '逸云佩',
        100: '莲音玦',
        120: '相思染',
        140: '玄龙苍珀',
        160: '碧海青天'
    },
    'bracelet': {
        60: '香木镯',
        80: '翡玉镯',
        100: '墨影扣',
        120: '花映月',
        140: '金水菩提',
        160: '浮雪幻音'
    }
}

MAIN_PROP = {
    'ring': ['伤害', '防御'],
    'earring': ['法术伤害', '法术防御'],
    'accessory': ['速度'],
    'bracelet': ['封印命中等级', '抵抗封印等级']
}

MAIN_PROP_VAL = {
    'ring': [0.2, 0.3],
    'earring': [0.2, 0.3],
    'accessory': [0.1, 6, 0.15, 9],
    'bracelet': [0.2, 0.3]
}

ADDITION_PROP = {
    'ring': ['伤害', '法术伤害', '固定伤害', '速度', '物理暴击等级', '法术暴击等级', '法术伤害结果', '治疗能力', '封印命中等级', '狂暴等级', '穿刺等级'],
    'earring': ['伤害', '法术伤害', '固定伤害', '速度', '物理暴击等级', '法术暴击等级', '法术伤害结果', '治疗能力', '封印命中等级', '狂暴等级', '穿刺等级'],
    'accessory': ['气血', '防御', '法术防御', '格挡值', '气血回复效果', '抵抗封印等级', '抗物理暴击等级', '抗法术暴击等级'],
    'bracelet': ['气血', '防御', '法术防御', '格挡值', '气血回复效果', '抵抗封印等级', '抗物理暴击等级', '抗法术暴击等级']
}

ADDITION_CATE = [
    ['速度', '法术伤害结果', '治疗能力', '狂暴等级', '穿刺等级'],
    ['伤害', '法术伤害', '固定伤害', '物理暴击等级', '法术暴击等级', '封印命中等级', '气血回复效果'],
    ['防御', '法术防御', '格挡值', '抵抗封印等级', '抗物理暴击等级', '抗法术暴击等级'],
    ['气血']
]

ADDITION_PROP_VAL = [
    [0.05, 3],
    [0.1, 0.15],
    [0.2, 0.3],
    [0.7, 1.05],
]

SPECIAL_EFFECT = ['超级简易']

P_SPECIAL_EFFECT = 0.05
P_ADD_3 = 0.3
NORMAL_SCALE_FACTOR = 0.2

2. Define data distribution.

In [2]:
import numpy as np


def dist(low, high, type):
    dist = {
        'uniform': uniform,
        'normal': normal
    }
    return dist[type](low, high)


def uniform(low, high):
    return int(np.random.uniform(low, high))

def normal(low, high):
    scale = low * NORMAL_SCALE_FACTOR
    val = np.random.normal(loc=low, scale=scale)
    if val < low:
        val = low
    elif val > high:
        val = high
    return int(val)


3. Define the key attribute generator.

In [3]:
import random


def generate_main_prop(level, type, distribution='normal'):
    prop_type = random.choice(MAIN_PROP[type])
    if type == 'accessory':
        low = MAIN_PROP_VAL[type][0] * level + MAIN_PROP_VAL[type][1]
        high = MAIN_PROP_VAL[type][2] * level + MAIN_PROP_VAL[type][3]
    else:
        low = MAIN_PROP_VAL[type][0] * level
        high = MAIN_PROP_VAL[type][1] * level
    prop_val = dist(low, high, distribution)
    return prop_type, prop_val

def generate_addition_prop(level, type, p_add3=P_ADD_3, distribution='normal'):
    add_list = []

    add1_type = random.choice(ADDITION_PROP[type])
    add1_val = generate_addition_val(level, add1_type, distribution)
    add_list.append((add1_type, add1_val))

    add2_type = random.choice(ADDITION_PROP[type])
    add2_val = generate_addition_val(level, add2_type, distribution)
    add_list.append((add2_type, add2_val))

    if np.random.random() < p_add3:
        add3_type = random.choice(ADDITION_PROP[type])
        add3_val = generate_addition_val(level, add3_type, distribution)
        add_list.append((add3_type, add3_val))
    
    return add_list

def generate_addition_val(level, add_type, distribution='normal'):
    if add_type in ADDITION_CATE[0]:
        low = ADDITION_PROP_VAL[0][0] * level + ADDITION_PROP_VAL[0][1]
        if level in [60, 80]:
            high = low + 3
        elif level in [100, 120]:
            high = low + 4
        else:
            high = low + 5
    else:
        for i in range(1, 4):
            if add_type in ADDITION_CATE[i]:
                low = ADDITION_PROP_VAL[i][0] * level
                high = ADDITION_PROP_VAL[i][1] * level
    add_val = dist(low, high, distribution)
    return add_val

def generate_special_effect():
    se = None
    if np.random.random() < P_SPECIAL_EFFECT:
        se = random.choice(SPECIAL_EFFECT)
    return se


4. Define accessory classes.

In [4]:
class lingshi_base(object):
    def __init__(self, level, type):
        self.level = level
        self.type = type
        self.pic = PIC_DIC[self.type][self.level]
        self.name = NAME_DIC[self.type][self.level]
        self.main_prop = None
        self.main_val = 0
        self.addition_prop = []
        self.special_effect = None

    def authenticate(self):
        self.special_effect = generate_special_effect()
        self.main_prop, self.main_val = generate_main_prop(self.level, self.type)
        self.addition_prop = generate_addition_prop(self.level, self.type)

    def print(self):
        print(f'{self.level}级{NAME[self.type]}: {self.name}')
        print(self.main_prop, self.main_val)
        if self.special_effect is not None:
            print(f'特效: {self.special_effect}')
        for i in range(len(self.addition_prop)):
            print(self.addition_prop[i])


class ring(lingshi_base):
    def __init__(self, level):
        assert level in LEVEL_POOL
        super().__init__(level, type='ring')


class earring(lingshi_base):
    def __init__(self, level):
        assert level in LEVEL_POOL
        super().__init__(level, type='earring')


class accessory(lingshi_base):
    def __init__(self, level):
        assert level in LEVEL_POOL
        super().__init__(level, type='accessory')


class bracelet(lingshi_base):
    def __init__(self, level):
        assert level in LEVEL_POOL
        super().__init__(level, type='bracelet')

5. Have fun!

In [8]:
r = ring(140)
r.authenticate()
r.print()

140级戒指: 九曜光华
伤害 33
('法术伤害结果', 10)
('法术暴击等级', 14)
